## Step 1 - Data Cleaning prior to Modeling ##
This project is based off of the Kaggle Competition: https://www.kaggle.com/c/home-credit-default-risk/overview

I will clean and prepare the dataset for modeling in this Notebook

Setting up the imports

In [1]:
# BASIC & PANDAS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# SKLEARN & MODELING
from sklearn.preprocessing import LabelEncoder

import gc; gc.enable()
import warnings
warnings.filterwarnings("ignore")

Loading the base dataset

In [2]:
df = pd.read_csv("../CSVs/Base_Train.csv")

Since the dataset is rather large to make it easier to work with I am going to break it into smaller chunks

In [3]:
# df.set_index('SK_ID_CURR',inplace=True)

In [4]:
#Breaking up each smaller dataset by 20 columns each
df2 = df[df.columns[1:21]]
df3 = df[df.columns[22:42]]
df4 = df[df.columns[43:63]]
df5 = df[df.columns[64:84]]
df6 = df[df.columns[85:105]]
df7 = df[df.columns[106:121]]

Now to check the null values in each of the smaller datasets

*All fixes have been checked previously using different dataframe cleaning methods*

Below are also columns which needed to be encoded

# Columns Dropped: #

* df3 - EXT_SOURCE_1 & 2 is being dropped due to it being an unknown feature
* df4-7 - The information in these dataframes are mostly personal features so for my project I am going to skip them

In [5]:
df3.drop(columns=['EXT_SOURCE_1'],inplace=True)

In [6]:
#For the columns with small portion as null values I am going to be replacing them with their medium values.
df2['AMT_ANNUITY'].fillna(df2['AMT_ANNUITY'].median(),inplace=True)
df2['AMT_GOODS_PRICE'].fillna(df2['AMT_GOODS_PRICE'].median(),inplace=True)
df2['NAME_TYPE_SUITE'].fillna('Unaccompanied',inplace=True)
df3['OCCUPATION_TYPE'].fillna('Laborers',inplace=True)
df3['CNT_FAM_MEMBERS'].fillna(df3['CNT_FAM_MEMBERS'].median(),inplace=True)

In [7]:
le = LabelEncoder() #Encoding the string values into numbers

df2['NAME_CONTRACT_TYPE'] = le.fit_transform(df2['NAME_CONTRACT_TYPE'])
df2['CODE_GENDER'] = le.fit_transform(df2['CODE_GENDER'])
df2['FLAG_OWN_CAR'] = le.fit_transform(df2['FLAG_OWN_CAR'])
df2['FLAG_OWN_REALTY'] = le.fit_transform(df2['FLAG_OWN_REALTY'])
df2['NAME_TYPE_SUITE'] = le.fit_transform(df2['NAME_TYPE_SUITE'])
df2['NAME_INCOME_TYPE'] = le.fit_transform(df2['NAME_INCOME_TYPE'])
df2['NAME_EDUCATION_TYPE'] = le.fit_transform(df2['NAME_EDUCATION_TYPE'])
df2['NAME_FAMILY_STATUS'] = le.fit_transform(df2['NAME_FAMILY_STATUS'])
df2['NAME_HOUSING_TYPE'] = le.fit_transform(df2['NAME_HOUSING_TYPE'])

df3['OCCUPATION_TYPE'] = le.fit_transform(df3['OCCUPATION_TYPE'])
df3['WEEKDAY_APPR_PROCESS_START'] = le.fit_transform(df3['WEEKDAY_APPR_PROCESS_START'])
df3['ORGANIZATION_TYPE'] = le.fit_transform(df3['ORGANIZATION_TYPE'])

Loading in the 2nd Dataset which has the information of previous loans that each individual has had

In [8]:
df_app = pd.read_csv("../CSVs/previous_application.csv")

In [9]:
#Pulling out the data from the dataset
df_apps = df_app[['SK_ID_CURR','AMT_APPLICATION']]
df_apps['AMT_OLD_ANNUITY']=df_app['AMT_ANNUITY']
df_apps['AMT_OLD_CREDIT']=df_app['AMT_CREDIT']

In [10]:
#Filling in the null values with the median values since most of the data in these columns are skewed towards the median
df_apps['AMT_OLD_ANNUITY'].fillna(df_apps['AMT_OLD_ANNUITY'].median(),inplace=True)
df_apps['AMT_OLD_CREDIT'].fillna(df_apps['AMT_OLD_CREDIT'].median(),inplace=True)

In [27]:
df_apps.drop_duplicates('SK_ID_CURR',keep='first',inplace=True)

Loading in the 3rd Dataset which has the Monthly balance snapshots of previous point of sales and cash loans that the applicant had with Home Credit.

In [11]:
df_pay = pd.read_csv("../CSVs/POS_CASH_balance.csv")

In [12]:
#Pulling out the data from the dataset
df_payment = df_pay[['SK_ID_CURR','CNT_INSTALMENT','CNT_INSTALMENT_FUTURE']]
# df_payment.set_index('SK_ID_CURR',inplace=True)

In [13]:
#Filling in the null values with the median values since most of the data in these columns are skewed towards the median
df_payment['CNT_INSTALMENT'].fillna(df_payment['CNT_INSTALMENT'].median(),inplace=True)
df_payment['CNT_INSTALMENT_FUTURE'].fillna(df_payment['CNT_INSTALMENT_FUTURE'].median(),inplace=True)

In [28]:
df_payment.drop_duplicates('SK_ID_CURR',keep='first',inplace=True)

Combining the Datasets into my final dataset

In [43]:
#Adding back in the ID to match up with the rest of the datasets and the target value
df2['SK_ID_CURR']=df['SK_ID_CURR']
df2['TARGET'] = df['TARGET']
df3['SK_ID_CURR']=df['SK_ID_CURR']

In [44]:
#Combining all the datasets
df_final = df2.merge(df3,how='left',on='SK_ID_CURR')
df_final2 = df_final.merge(df_apps,how='left',on='SK_ID_CURR')
df_final3 = df_final2.merge(df_payment,how='left',on='SK_ID_CURR')

In [46]:
#Since there were IDs that were not present in the apps and payment datasets I am going to drop the nulls
#So I can work with a dataset with every value filled in
df_final3.dropna(inplace=True)

In [47]:
df_final3

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,AMT_APPLICATION,AMT_OLD_ANNUITY,AMT_OLD_CREDIT,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE
0,1,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,5,179055.0,9251.775,179055.0,24.0,18.0
1,0,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,39,900000.0,98356.995,1035882.0,12.0,1.0
2,0,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,11,24282.0,5357.250,20106.0,4.0,2.0
3,0,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,5,675000.0,24246.000,675000.0,5.0,0.0
4,0,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,0,1,1,37,180000.0,16509.600,180000.0,24.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0,1,0,0,0,157500.0,254700.0,27558.0,225000.0,...,0,0,0,0,43,40455.0,6605.910,40455.0,12.0,7.0
307507,0,0,0,0,1,0,72000.0,269550.0,12001.5,225000.0,...,0,0,0,0,57,57595.5,10074.465,56821.5,6.0,0.0
307508,0,0,0,0,1,0,153000.0,677664.0,29979.0,585000.0,...,0,0,1,1,39,28912.5,5567.715,27306.0,6.0,2.0
307509,1,0,0,0,1,0,171000.0,370107.0,20205.0,319500.0,...,0,1,1,0,3,223789.5,19065.825,247423.5,16.0,14.0


Finally to make the csv file we will use for the modeling process

In [48]:
df_final3.to_csv("../CSVs/Model_Ready_Data.csv")